In [1]:
#import dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
#read in merged and cleaned data
df = pd.read_csv('../../data/merged_cleaned_data/cases_merged_data.csv')
df

,Unnamed: 0,FIPS,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,...,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,first_year_cases
0,0,1001,594.443459,55200,23315,21115,8422,1065,29372,4204,...,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,6589
1,1,1009,644.830460,57645,24222,20600,8220,909,22656,7861,...,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,6444
2,2,1013,776.838201,20025,10026,6708,4640,567,20430,2141,...,19011.0,78.563680,76.109761,76.623924,69.058104,79.956648,67.920284,72.773953,41.789629,2097
3,3,1015,605.867251,115098,53682,45033,20819,4628,24706,12620,...,22231.0,79.439032,79.955121,77.918741,54.063568,76.745724,67.456150,68.292794,29.845243,14224
4,4,1017,596.560643,33826,16981,13516,5531,773,22827,4383,...,21532.0,76.995358,78.156771,75.891100,67.343775,79.128558,66.397785,69.554441,41.644857,3488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,3053,48131,1793.476183,11355,5592,3511,2751,482,17864,2386,...,19853.0,79.125428,78.895880,76.629575,60.576045,73.670302,64.571017,68.007770,50.959826,1214
3054,3054,48505,998.411980,14369,6388,4405,5609,621,17228,3226,...,16007.0,79.355639,79.572483,74.378252,77.443239,76.386871,74.001471,73.609838,47.134744,1760
3055,3055,48507,1297.406535,12131,4344,3509,4150,421,13350,2719,...,13393.0,78.392216,76.024682,75.848196,76.967659,77.303576,70.010162,71.121990,65.403060,1844
3056,3056,48127,1328.884075,10663,4408,3309,3148,694,18121,2137,...,19528.0,79.457964,79.141556,66.677924,73.140121,71.659882,68.065499,66.378831,61.756683,1707


In [4]:
#set index to FIPS
df = df.set_index(df['FIPS'])
df= df.drop(columns = ['FIPS','Unnamed: 0'])

In [5]:
#create variable for case% for each counties population
df['case_pct'] = df['first_year_cases']/df['E_TOTPOP']*100
df['case_pct'].head()

FIPS
1001    11.936594
1009    11.178767
1013    10.471910
1015    12.358164
1017    10.311595
Name: case_pct, dtype: float64

In [6]:
df['case_pct'].describe()

count    3058.000000
mean        9.426600
std         3.045809
min         0.000000
25%         7.713422
50%         9.466675
75%        11.176131
max        38.010657
Name: case_pct, dtype: float64

In [7]:
# bin and cut the case_pct column into 2 classifications
q = df['case_pct'].quantile(.9)
bins = [0, q , 40]
labels = ['low','high']
df['case_class'] = pd.cut(df['case_pct'], bins, labels = labels)
df['case_class']

FIPS
1001      low
1009      low
1013      low
1015      low
1017      low
         ... 
48131     low
48505     low
48507    high
48127    high
48247     low
Name: case_class, Length: 3058, dtype: category
Categories (2, object): ['low' < 'high']

In [8]:
df['case_class'].value_counts()

low     2728
high     306
Name: case_class, dtype: int64

In [9]:
#drop unneeded columns
df = df.drop('case_pct', axis =1)
df

,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,E_AGE17,...,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,first_year_cases,case_class
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,594.443459,55200,23315,21115,8422,1065,29372,4204,8050,13369,...,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,6589,low
1009,644.830460,57645,24222,20600,8220,909,22656,7861,10233,13468,...,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,6444,low
1013,776.838201,20025,10026,6708,4640,567,20430,2141,3806,4566,...,78.563680,76.109761,76.623924,69.058104,79.956648,67.920284,72.773953,41.789629,2097,low
1015,605.867251,115098,53682,45033,20819,4628,24706,12620,19386,25196,...,79.439032,79.955121,77.918741,54.063568,76.745724,67.456150,68.292794,29.845243,14224,low
1017,596.560643,33826,16981,13516,5531,773,22827,4383,6409,7006,...,76.995358,78.156771,75.891100,67.343775,79.128558,66.397785,69.554441,41.644857,3488,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,1793.476183,11355,5592,3511,2751,482,17864,2386,2025,2962,...,79.125428,78.895880,76.629575,60.576045,73.670302,64.571017,68.007770,50.959826,1214,low
48505,998.411980,14369,6388,4405,5609,621,17228,3226,1999,4835,...,79.355639,79.572483,74.378252,77.443239,76.386871,74.001471,73.609838,47.134744,1760,low
48507,1297.406535,12131,4344,3509,4150,421,13350,2719,1665,3583,...,78.392216,76.024682,75.848196,76.967659,77.303576,70.010162,71.121990,65.403060,1844,high


In [10]:
#turn case % classifications into binary 
df = pd.get_dummies(df, columns = ['case_class'])
df

,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,E_AGE17,...,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,first_year_cases,case_class_low,case_class_high
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,594.443459,55200,23315,21115,8422,1065,29372,4204,8050,13369,...,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,6589,1,0
1009,644.830460,57645,24222,20600,8220,909,22656,7861,10233,13468,...,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,6444,1,0
1013,776.838201,20025,10026,6708,4640,567,20430,2141,3806,4566,...,76.109761,76.623924,69.058104,79.956648,67.920284,72.773953,41.789629,2097,1,0
1015,605.867251,115098,53682,45033,20819,4628,24706,12620,19386,25196,...,79.955121,77.918741,54.063568,76.745724,67.456150,68.292794,29.845243,14224,1,0
1017,596.560643,33826,16981,13516,5531,773,22827,4383,6409,7006,...,78.156771,75.891100,67.343775,79.128558,66.397785,69.554441,41.644857,3488,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,1793.476183,11355,5592,3511,2751,482,17864,2386,2025,2962,...,78.895880,76.629575,60.576045,73.670302,64.571017,68.007770,50.959826,1214,1,0
48505,998.411980,14369,6388,4405,5609,621,17228,3226,1999,4835,...,79.572483,74.378252,77.443239,76.386871,74.001471,73.609838,47.134744,1760,1,0
48507,1297.406535,12131,4344,3509,4150,421,13350,2719,1665,3583,...,76.024682,75.848196,76.967659,77.303576,70.010162,71.121990,65.403060,1844,0,1


In [11]:
#seperate targets and features
X = df.drop(['case_class_low','case_class_high'], axis=1).values
y=df['case_class_high'].values

In [12]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
# hidden_nodes_layer1 = 100
# hidden_nodes_layer2 = 50
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 100
# hidden_nodes_layer3 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 150)               15600     
                                                                 
 dense_1 (Dense)             (None, 100)               15100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 30,801
Trainable params: 30,801
Non-trainable params: 0
_________________________________________________________________


2022-08-10 14:34:50.056189: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
72/72 [==============================] - 1s 2ms/step - loss: 599.1625 - accuracy: 0.8338
Epoch 2/100
72/72 [==============================] - 0s 5ms/step - loss: 209.5246 - accuracy: 0.8247
Epoch 3/100
72/72 [==============================] - 0s 3ms/step - loss: 83.8288 - accuracy: 0.8666
Epoch 4/100
72/72 [==============================] - 0s 2ms/step - loss: 351.0917 - accuracy: 0.8491
Epoch 5/100
72/72 [==============================] - 0s 2ms/step - loss: 159.9258 - accuracy: 0.8482
Epoch 6/100
72/72 [==============================] - 0s 3ms/step - loss: 72.3728 - accuracy: 0.8591
Epoch 7/100
72/72 [==============================] - 0s 2ms/step - loss: 170.0905 - accuracy: 0.8757
Epoch 8/100
72/72 [==============================] - 0s 2ms/step - loss: 360.6897 - accuracy: 0.8539
Epoch 9/100
72/72 [==============================] - 0s 2ms/step - loss: 97.3474 - accuracy: 0.8556
Epoch 10/100
72/72 [==============================] - 0s 2ms/step - loss: 188.8599 - accuracy:

72/72 [==============================] - 0s 2ms/step - loss: 0.5271 - accuracy: 0.9429
Epoch 83/100
72/72 [==============================] - 0s 2ms/step - loss: 0.3585 - accuracy: 0.9551
Epoch 84/100
72/72 [==============================] - 0s 2ms/step - loss: 0.3963 - accuracy: 0.9525
Epoch 85/100
72/72 [==============================] - 0s 2ms/step - loss: 0.2825 - accuracy: 0.9485
Epoch 86/100
72/72 [==============================] - 0s 2ms/step - loss: 0.2622 - accuracy: 0.9560
Epoch 87/100
72/72 [==============================] - 0s 2ms/step - loss: 1.9643 - accuracy: 0.9341
Epoch 88/100
72/72 [==============================] - 0s 2ms/step - loss: 16.1870 - accuracy: 0.8818
Epoch 89/100
72/72 [==============================] - 0s 2ms/step - loss: 24.3953 - accuracy: 0.8657
Epoch 90/100
72/72 [==============================] - 0s 2ms/step - loss: 9.8287 - accuracy: 0.8657
Epoch 91/100
72/72 [==============================] - 0s 3ms/step - loss: 1.9919 - accuracy: 0.9041
Epoch 92/10

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

25/25 - 0s - loss: 11.3262 - accuracy: 0.8948 - 248ms/epoch - 10ms/step
Loss: 11.326242446899414, Accuracy: 0.8948051929473877
